## Imports

In [1]:
%matplotlib inline
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from components.AttractorLayer import AttractorLayer
from components.RadarLoader import RadarLoader

seq_name = "2020-02-28-13-13-43"
%run carrada_utils/scripts/set_path.py 'components/carrada_datasets/'

## Set Parameters

In [2]:
np.zeros((3*3))

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [3]:
TAU = 1  # TODO: I don't know whether this is important
INTENSITY =1.0
BETA =300  # TODO : Check this
SIGMA = 3.0 # TODO: Check this
SHIFT = 0.0 # TODO : Check this
CUTOFF_DIST = 10 # TODO : Change this
X_EYE = 1.0
Y_EYE = 1.0
NR_OF_PASSES = 4

K_INHIB = 5.0

In [ ]:
data_loader = RadarLoader(seq_name)
data, size_bf = data_loader.get_range_angle_stream_data(clip_and_normalize=True, resize=(64,64))
data_diff_normalized = data_loader.get_range_angle_stream_data_differentiated(clip_and_normalize=True, resize=(64,64))
raw_camera_data = data_loader.get_color_image_datastream(resize=(64,64))

# Load parameters and network

In [ ]:
attr_layer = AttractorLayer(
        tau=TAU,
        intensity=INTENSITY,
        cutoff_dist=CUTOFF_DIST,
        sigma=SIGMA,
        shift=SHIFT,
        beta=BETA,
        k=K_INHIB,
        clip=True,
        x_eye=X_EYE,
        y_eye=Y_EYE,
)

attr_layer.set_weights()
attr_layer.save_network(file_path="components/network_weights/threes_win.npy")

#attr_layer.load_network(file_path="components/network_weights/threes_win.npy")

# Run process

In [ ]:
neuron_act = []
neuron_act_final = []
for data_entry in data_diff_normalized:
    # Forward pass
    activities_step = attr_layer.forward_pass_visualization(data_entry.flatten(), number_of_passes=NR_OF_PASSES)
    # Get neuron activities
    neuron_act.append(activities_step[0].copy())
    neuron_act_final.append(activities_step[NR_OF_PASSES-1].copy())


In [ ]:
IO_CUTOFF = 0.4
output_ = []
for i in range(0,len(neuron_act)):
    # Use input and activities to get output
    data_entry_clipped = neuron_act_10[i].copy()
    activities_step_clipped = data_diff_normalized[i].copy()
    #data_entry_clipped[data_entry_clipped < 0.1] = 0.0
    #activities_step_clipped[activities_step_clipped < 0.1] = 0.0
    output_step =  data_entry_clipped * activities_step_clipped
    output_step[output_step < 0.001] = 0.0
    output_step[output_step > 0.0009] = 1.0
    output_.append(output_step.copy())
    

# Get annotations and calculate score

In [ ]:
dense_, sparse_, box_, sp_mp_, sp_mp_vis_ = data_loader.visualize_annotations(differentiated=True, size_bf=size_bf)

In [ ]:
ground_truth_annot = []
predicted_location_network = []

difference_in_location = []
it_i = 0
for data_entry in sp_mp_:
    s_ = np.zeros((64,64))
    tt_ = np.zeros((64,64))
    for sub_entry in data_entry:
        s_[sub_entry[0] // 4][sub_entry[1] // 4] = 1
        loc_ = np.argmax(neuron_act_10[it_i])
        tt_[loc_// 64][loc_ %64]= 1
        
        diff_loc_ = np.sqrt(((loc_// 64) - sub_entry[0]//4)**2 + ((loc_ %64) - sub_entry[1]//4)**2)
        
        difference_in_location.append(diff_loc_)
        break
        
    predicted_location_network.append(tt_.copy())    
    ground_truth_annot.append(s_.copy())
    it_i +=1

In [ ]:
plt.plot(difference_in_location)

In [ ]:
CUTOFF = 0.5
mIoU_dense = []
mDice_dense = []
mIoU_sparse = []
mDice_sparse = []
for i in range(0,len(dense_)):
    A_acts = neuron_act[i].copy()
    A_acts[A_acts < 0.1] = 0
    A_acts[A_acts > 0.09] = 1
    B_gt_dense = dense_[i].copy() * 1
    B_gt_sparse = sparse_[i].copy() * 1
     
    A_B_intersection_dense = A_acts.copy()
    A_B_intersection_dense[B_gt_dense == 1.0] = 1.0
    A_B_intersection_dense[B_gt_dense != 1.0] = 0.0
    
    A_B_union_dense = B_gt_dense.copy()
    A_B_union_dense[A_acts > 0] = 1.0
    
    if np.sum(A_B_union_dense) <=0.001:
        continue
    mIoU_dense.append(np.sum(A_B_intersection_dense) / np.sum(A_B_union_dense))
    mDice_dense.append(np.sum(A_B_intersection_dense) * 2 / (np.sum(B_gt_dense) + np.sum(A_acts)))
    
     
    A_B_intersection_sparse = A_acts.copy()
    A_B_intersection_sparse[B_gt_sparse == 1.0] = 1.0
    A_B_intersection_sparse[B_gt_sparse != 1.0] = 0.0
    
    A_B_union_sparse = B_gt_sparse.copy()
    A_B_union_sparse[A_acts > 0] = 1.0
    if np.sum(A_B_union_sparse) <=0.001:
        continue
    mIoU_sparse.append(np.sum(A_B_intersection_sparse) / np.sum(A_B_union_sparse))
    mDice_sparse.append(np.sum(A_B_intersection_sparse) * 2 / (np.sum(B_gt_sparse) + np.sum(A_acts)))
    
    
title_string = f"Losses \n        mIoU_dense: {np.mean(mIoU_dense[40:]):.6f}\n     mIoU_sparse: {np.mean(mIoU_sparse[40:]):.6f}\n"
print(title_string)

In [ ]:


fig,((ax1, ax2, ax6), (ax3, ax4, ax5))= plt.subplots(2, 3, figsize=(18,5), dpi=100)
fig.suptitle(title_string, fontsize=10)

# Ground Truth Comparison
gt_comparison = np.zeros((len(neuron_act), 64,64,3))
gt_comparison[:,:,:,1] = ground_truth_annot 
gt_comparison[:,:,:,2] = predicted_location_network

ax1.tick_params(axis='both',labelsize=8)
ax1.set_title("Ground Truth vs. Predicted Location",fontsize=10)
gt_comp_map = ax1.imshow(gt_comparison[0], cmap='hot', interpolation='none')

# Neuron Activities
ax2.tick_params(axis='both',labelsize=8)
ax2.set_title(f"Neuron Activities after {NR_OF_PASSES} passes",fontsize=10)
cell_act_map = ax2.imshow(np.array(neuron_act_final[0]), cmap='hot', interpolation='none')

# Annotated Neuron Activities
annotated_neuron_act = np.zeros((len(neuron_act), 64,64,3))
annotated_neuron_act[:,:,:,0] = neuron_act_final / np.max(neuron_act_final)
annotated_neuron_act[:,:,:,1] = dense_

ax6.tick_params(axis='both',labelsize=8)
ax6.set_title("Neuron Activities with Annotation",fontsize=10)
cell_act_map_w_annot = ax6.imshow(annotated_neuron_act[0], cmap='hot', interpolation='none')

# Differentiated Input for the data
ax3.tick_params(axis='both',labelsize=8)
ax3.set_title("Differentiated Input",fontsize=10)
input_diff_map = ax3.imshow(np.array(data_diff_normalized[0]), cmap='hot',interpolation='none')

# Raw data input
ax4.tick_params(axis='both',labelsize=8)
ax4.set_title("Raw Data Input",fontsize=10)
input_map = ax4.imshow(np.array(data[1]), cmap='hot', interpolation='none')

# Raw RGB Camera data
ax5.tick_params(axis='both',labelsize=8)
ax5.set_title("Raw Camera RGB Data",fontsize=10)
rgb_cam_data = ax5.imshow(np.array(raw_camera_data[1]), interpolation='none')]


def init():
    cell_act_map_w_annot.set_data(annotated_neuron_act[0])
    gt_comp_map.set_data(gt_comparison[0])
    cell_act_map.set_data(np.array(neuron_act[0]))
    input_diff_map.set_data(np.array(data_diff_normalized[0]))
    input_map.set_data(np.array(data[1]))
    rgb_cam_data.set_data(np.array(raw_camera_data[1]))
    return [cell_act_map_w_annot, cell_act_map, input_diff_map, input_map, rgb_cam_data]

# animation function.  This is called sequentially
def animate(i):
    cell_act_map_w_annot.set_data(annotated_neuron_act[i])
    gt_comp_map.set_data(gt_comparison[i])
    cell_act_map.set_data(np.array((neuron_act[i])))
    input_diff_map.set_data(np.array(data_diff_normalized[i]))
    input_map.set_data(np.array(data[i+1]))
    rgb_cam_data.set_data(np.array(raw_camera_data[i+1]))
    return [cell_act_map_w_annot, cell_act_map, input_diff_map, input_map, rgb_cam_data]


anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=150, interval=1, blit=True)

anim.save('animations/final_anim1_sp_cam_pass_10.gif', fps=10)

from IPython.display import Image, display
with open('animations/final_anim1_sp_cam_pass_10.gif','rb') as file:
    display(Image(file.read()))
    
    